### QUESTION 1

Text classification is an example of Na ̈ıve Bayes application. You are required to classify the
following statements, “a cup of hot coffee” and “a cone of ice cream”, given the categories Sunny
and Rainy.

Given a training data, an objective of Na ̈ıve Bayes will be to compute P(sunny|a cone of
ice cream) and P(rainy|a cup of hot coffee) and classify the statement as the category with a
higher probability.

1. P(sunny|a cone of ice cream) = ?
2. P(rainy|a cup of hot coffee) = ?

#### Answer:

Bayes Theorem states thus $ P(A|B) = \frac{P(B|A) * P(A)}{P(B)} $

we assume that our given probabilities are made up of independent events as follows:

$ P(a\, cone\, of\, ice\, cream\,) = P(a) * P(cone) * P(of) * P(ice) * P(cream) \quad $
$ P(a\, cup\, of\, hot\,\, coffee\,) = P(a) * P(cup) * P(of) * P(hot) * P(coffee) $

the probability of each independent event will be as follows:

$ P(sunny\,|\,a\, cone\, of\, ice\, cream\,) = P(a|sunny) * P(cone|sunny) * P(of|sunny) * P(ice|sunny) * P(cream|sunny) $
$ P(rainy\,|\,a \, cup\, of\, hot\,\, coffee\,) = P(a|rainy) * P(cup|rainy) * P(of|rainy) * P(hot|rainy) * P(coffee|rainy) $


### QUESTION 2
*Implementing the Naive Bayes Classifier*

In [1]:
import util
import classificationMethod
import math

class NaiveBayesClassifier(classificationMethod.ClassificationMethod):
    """
    See the project description for the specifications of the Naive Bayes classifier.
  
    Note that the variable 'datum' in this code refers to a counter of features
    (not to a raw samples.Datum).
    """
    def __init__(self, legalLabels):
        self.legalLabels = legalLabels
        self.type = "naivebayes"
        self.k = 1 # this is the smoothing parameter, ** use it in your train method **
        self.automaticTuning = False # Look at this flag to decide whether to choose k automatically ** use this in your train method **
    
    def setSmoothing(self, k):
        """
        This is used by the main method to change the smoothing parameter before training.
        Do not modify this method.
        """
        self.k = k

    def train(self, trainingData, trainingLabels, validationData, validationLabels):
        """
        Outside shell to call your method. Do not modify this method.
        """  
      
        self.features = list(trainingData[0].keys()) # this could be useful for your code later...
    
        if (self.automaticTuning):
            kgrid = [0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 20, 50]
        else:
            kgrid = [self.k]
        
        self.trainAndTune(trainingData, trainingLabels, validationData, validationLabels, kgrid)
      
    def trainAndTune(self, trainingData, trainingLabels, validationData, validationLabels, kgrid):
        """
        Trains the classifier by collecting counts over the training data, and
        stores the Laplace smoothed estimates so that they can be used to classify.
        Evaluate each value of k in kgrid to choose the smoothing parameter 
        that gives the best accuracy on the held-out validationData.
    
        trainingData and validationData are lists of feature Counters.  The corresponding
        label lists contain the correct label for each datum.
    
        To get the list of all possible features or labels, use self.features and 
        self.legalLabels.
        """

        "*** YOUR CODE HERE ***"
        # Initialize best accuracy to 0
        bestAccuracy = 0

        Prob = util.Counter()
        for i in trainingLabels:
            Prob[i] += 1
        Prob.normalize()
        self.post = Prob

        # Initialize stuff
        counts = {}
        totals = {}
        for feat in self.features:
            counts[feat] = {0: util.Counter(), 1: util.Counter()}
            totals[feat] = util.Counter()

        # Calculate totals and counts
        for dat, datum in enumerate(trainingData):
            tot = trainingLabels[dat]
            for feat, value in list(datum.items()):
                counts[feat][value][tot] += 1.0
                totals[feat][tot] += 1.0 

        bestConditionals = {}
    
        # Evaluate each k, and use the one that yields the best accuracy
        for k in kgrid or [0.0]:
            correct = 0
            conditionals = {}            
            for feat in self.features:
                conditionals[feat] = {0: util.Counter(), 1: util.Counter()}

            # Run Laplace smoothing
            for feat in self.features:
                for value in [0, 1]:
                    for label in self.legalLabels:
                        conditionals[feat][value][label] = (counts[feat][value][label] + k) / (totals[feat][label] + k * 2)

            # Check the accuracy associated with this k
            self.conditionals = conditionals              
            guesses = self.classify(validationData)
            for label, guess in enumerate(guesses):
                correct += (validationLabels[label] == guess and 1.0 or 0.0)
            accuracy = correct / len(guesses)

            # Keep the best k so far
            # if accuracy > bestAccuracy or bestAccuracy is None:
            if accuracy > bestAccuracy:
                bestAccuracy = accuracy
                bestConditionals = conditionals
                self.k = k

        self.conditionals = bestConditionals
        # util.raiseNotDefined()
        
    def classify(self, testData):
        """
        Classify the data based on the posterior distribution over labels.
    
        You shouldn't modify this method.
        """
        guesses = []
        self.posteriors = [] # Log posteriors are stored for later data analysis (autograder).
        for datum in testData:
            posterior = self.calculateLogJointProbabilities(datum)
            guesses.append(posterior.argMax())
            self.posteriors.append(posterior)
        return guesses

    def calculateLogJointProbabilities(self, datum):
        """
        Returns the log-joint distribution over legal labels and the datum.
        Each log-probability should be stored in the log-joint counter, e.g.    
        logJoint[3] = <Estimate of log( P(Label = 3, datum) )>
        """
        logJoint = util.Counter()

        "*** YOUR CODE HERE ***"

        evidence = datum.items()

        for label in self.legalLabels:
            logJoint[label] = math.log(self.post[label])
            for cond in self.conditionals:
                prob = self.conditionals[cond][datum[cond]][label]
                logJoint[label] += (prob and math.log(prob) or 0.0)
        # util.raiseNotDefined()

        return logJoint

    def findHighOddsFeatures(self, label1, label2):
        """
        Returns the 100 best features for the odds ratio:
                P(feature=1 | label1)/P(feature=1 | label2) 
        """
        featuresOdds = []

        "*** YOUR CODE HERE ***"
        for feature in self.features:
            featuresOdds.append((self.conditionals[feature, label1] / self.conditionals[feature, label2], feature))
            featuresOdds.sort()
            featuresOdds = [feat for val, feat in featuresOdds[-100:]]

        #util.raiseNotDefined()

        return featuresOdds


In [2]:
%run dataClassifier.py -c naiveBayes -a

Doing classification
--------------------
data:		digits
classifier:		naiveBayes
training set size:	100
using automatic tuning for naivebayes
trying to read: digitdata/trainingimages
testing
trying to read: digitdata/traininglabels
testing
trying to read: digitdata/validationimages
testing
trying to read: digitdata/validationlabels
testing
trying to read: digitdata/testimages
testing
trying to read: digitdata/testlabels
testing
Extracting features...
Training...
Validating...
('74', 'correct out of 100 (74.0%).')
Testing...
('65', 'correct out of 100 (65.0%).')
Mistake on example 3
Predicted 3; truth is 5
Image: 
                            
                            
                            
                            
                            
          +#########+       
         +###########+      
         ############+      
         ############       
         ####+++#####       
         +##+     +++       
         +###++++           
          ########+         
   

### References for Question No 2.

- http://ai.berkeley.edu/projects/release/classification/v1/001/docs/naiveBayes.html
- https://github.com/anthony-niklas/cs188/blob/master/p5/naiveBayes.py
- https://github.com/anthony-niklas/cs188/blob/341f854af50863f6f30e09ca32910ee3025ec5b2/p5/dataClassifier.py
- https://www.youtube.com/watch?v=FgaM-TzT7qk&feature=emb_imp_woyt